# AutoGluonでの回帰モデルとして解く

結局、時系列予測は回帰問題として解いた方が汎用性がありそう

その場合、特徴量生成が重要になる。

* (回帰) kaggleでの特徴量で、AutoGluonでとく
* (時系列)データそのままで、GluonTSでとく。特徴量の生成が大変だから。
* (時系列)データそのままで、AmazonForecastでとく
* (回帰) tsfreshの特徴量で、AutoGluonでとく（Kaggle特徴量との比較）

実業務では、「どの特徴量が重要なのか」を探っていくことが重要

## AutoGluonのインストール

https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html

SageMakerノートブックインスタンスにはインストールできない？ -> numbaとllvmliteに対して以下を実行

In [ ]:
!pip install -U numba
!pip install llvmlite --ignore-installed

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

In [ ]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

In [ ]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
predictor.leaderboard(test_data, silent=True)

In [ ]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=<variable-name>).fit(train_data=<file-name>)

In [ ]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)